# Mercari Price Suggestion Challenge:
> Dự đoán giá tiền của một loại mặt hàng dựa vào các thông số kĩ thuật, mô tả của người bán, hình thức bán...

Báo cáo vấn đề Có thể khó biết thứ gì đó thực sự đáng giá bao nhiêu. Chi tiết nhỏ có thể có nghĩa là sự khác biệt lớn về giá cả. Ví dụ, một trong những chiếc áo len có giá 335 đô la và chiếc còn lại có giá 9,99 đô la thì điều gì quyết định về giá của sản phẩm </br>

Việc định giá sản phẩm thậm chí còn khó hơn trên quy mô lớn, chỉ cần xem xét có bao nhiêu sản phẩm được bán trực tuyến. Quần áo có xu hướng định giá theo mùa mạnh mẽ và bị ảnh hưởng nhiều bởi thương hiệu, trong khi đồ điện tử có giá dao động dựa trên thông số kỹ thuật của sản phẩm. </br>

Mercari muốn đưa ra đề xuất về giá cho người bán, nhưng điều này rất khó vì người bán của họ được phép đưa bất kỳ thứ gì hoặc bất kỳ gói nào lên thị trường của Mercari.

# YÊU CẦU
Sử dụng các thông số về danh mục sản phẩm, thương hiệu, tình trạng mặt hàng,... để dự đoán giá của các mặt hàng được bán

**Mục tiêu** </br>
Dự đoán giá của một mặt hàng với tình trạng, mô tả và các tính năng liên quan khác. Giảm thiểu sự khác biệt giữa giá dự đoán và giá thực tế (RMSLE) </br>
**Đánh giá tổng quan:** </br>
Đây là bài toán hồi quy tuyến tính. Dữ liệu được cung cấp ở dạng text và lable, nên cần chuyển các đặc trưng về dạng số thực hoặc vector để xử lý tuyến tính

# Điều làm được trong lần test này (version4):
Nhận thấy loss vẫn còn ở tỉ số cao -> tăng thời gian huấn luyện
tăng epoch(5->20) Nhận thấy được loss giảm đi tương đối nhiều (0.0159->0.0112) 
Kết quả: Giảm được Error của hàm RMSLE từ 0.506 xuống 0.488

**Hạn chế:** Code còn lớn, chiếm dung lượng nhiều, xử lý chậm (mỗi epoch chạy trong khoảng 280s)

Chưa khắc phục được: Chưa tìm được model tốt hơn để thay thế (score vẫn giữ ở mức ~ 0.6) (best: 0.3888)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline 

import math


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z

Bước 1: Phân tích dữ liệu khám phá Bước đầu tiên để giải quyết bất kỳ nghiên cứu điển hình nào trong khoa học dữ liệu là xem xét và phân tích dữ liệu bạn có một cách chính xác. Nó giúp cung cấp những hiểu biết có giá trị về mô hình và thông tin mà nó phải truyền tải. </br> Để tải dữ liệu, chúng ta chỉ cần tệp train.tsv. Chúng tôi sẽ tải nó vào khung dữ liệu gấu trúc.

In [ ]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5
#Source: https://www.kaggle.com/marknagelberg/rmsle-function

In [ ]:
#Sử dụng pandas đọc file dữ liệu 
train = pd.read_table("train.tsv", sep="\t")
test = pd.read_csv("../input/mercari-price-suggestion-challenge/test_stg2.tsv.zip" , sep='\t')
print(train.shape)
print(test.shape)

In [ ]:
train.shape
train.head(5)

có một tập dữ liệu gồm 1482535 sản phẩm cùng với các thông số của chúng. 5 hàng đầu tiên xuất hiện như hình trên. </br> xem thêm một số thông tin về dữ liệu ở dưới

In [ ]:
train.info()


Kiểm tra các giá trị null trong tập dữ liệu: </br> Người ta phải kiểm tra các giá trị bị thiếu trong tập dữ liệu trước khi sử dụng bất kỳ mô hình học máy nào. </br> Sau đó ta sẽ phải "lấp" các giá trị trống của chúng

In [ ]:
train.isnull().sum()

Chúng ta có thể thấy rằng các cột ‘category_name’, ‘brand_name’ và ‘item_description’ có giá trị rỗng. Có các kỹ thuật khác nhau để xử lý các giá trị bị thiếu như loại bỏ các hàng có giá trị bị thiếu, loại bỏ đối tượng địa lý có tỷ lệ giá trị bị thiếu cao hoặc lấp đầy các cột có giá trị bị thiếu bằng một số giá trị khác. Ở đây, chúng ta đã chọn để lấp đầy các giá trị còn thiếu này trong dữ liệu bằng một số giá trị khác.

In [ ]:
def handle_missing(dataset):
    #xử lý dữ liệu trống
    dataset.category_name.fillna(value="missing", inplace=True)
    dataset.brand_name.fillna(value="missing", inplace=True)
    dataset.item_description.fillna(value="missing", inplace=True)
    return (dataset)

In [ ]:
train = handle_missing(train)
test = handle_missing(test)

# Phân tích số liệu </br>
**Price** </br>
Dưới đây là sự phân bố các mức giá được thống kê

In [ ]:
train.price.describe()

Vì vậy, từ bảng mô tả, chúng ta có thể kết luận rằng: </br> 25% sản phẩm có giá dưới 10 $, 50% sản phẩm có giá dưới 17 $ và 75% sản phẩm có giá dưới 29 $ 

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(14,8))
ax.hist(train.price,bins = 30, range = [0,200],label="Price")
ax.set_xlabel('Price',fontsize=15)
ax.set_ylabel('No of items', fontsize=15)
plt.show()

Ta nhận thấy sự phân phối về giá bị lệch rất nhiều </br> 
Lỗi này là Lỗi lôgarit bình phương gốc (RMSLE).Do đó, áp dụng phép biến đổi logarit cho biến mục tiêu giá, để làm cho giả định này có sẵn cho việc đào tạo mô hình.

In [ ]:
train["logPrice"] = np.log(train["price"]+1)

In [ ]:
from scipy.stats import norm
import seaborn as sns


fig, axes = plt.subplots(1, 3, figsize=(10,4))

axes[0].set_title('Price')
sns.distplot(train.price, ax=axes[0], kde=False)
axes[0].grid()

axes[1].set_title('Price < 75')
sns.distplot(train.price[train.price<75], ax=axes[1], kde=False)
axes[1].grid()

axes[2].set_title('log(Price + 1)')
sns.distplot(train["logPrice"], ax=axes[2], fit=norm, kde=False)
axes[2].set_xticks(range(0,9))
axes[2].grid()

Qua so sánh 3 biểu đồ ta có thể thấy dữ liệu khi chuyển đổi logarit có sự ổn định hơn (chuẩn hơn)


In [ ]:
def plot_distribution_and_violin(variable):
    fig, axes = plt.subplots(2,1,figsize=(5,6), sharex=True)
    axes[0].set_title(variable)
    sns.countplot(x=variable, data=train, palette="ch:.25", color="c", ax=axes[0])
    sns.violinplot(x=variable, y='logPrice', palette="ch:.25", data=train, ax=axes[1])
    fig.tight_layout()

plot_distribution_and_violin('shipping')
print('The shipping prices of products are either ‘1’(buyer charged) or ‘0’(seller charged)')

Ở đây, chúng ta có thể thấy rằng đối với các mặt hàng có giá thấp hơn, người mua thường phải trả phí vận chuyển vì lý do lợi nhuận. Ngoài ra, khi giá tăng lên, chúng ta có thể thấy rằng người bán đã lựa chọn thanh toán phí vận chuyển để kích thích tiêu dùng. Xu hướng này thường được quan sát thấy khi mua sản phẩm trực tuyến với giá trị thấp hơn một ngưỡng nhất định để được giao hàng miễn phí.

In [ ]:
train['category_name'].value_counts()[:10]


Từ số liệu thống kê, có thể nói, Women có số lượng mặt hàng tối đa. </br>
Tên danh mục được liệt kê bằng dấu phân cách ‘/’ cho biết main category, sub-category 1 và sub-category 2 của sản phẩm. Do đó, để hiểu rõ hơn về từng sản phẩm, ta sẽ thực hiện kỹ thuật chia tên danh mục thành 3 cột khác nhau, cụ thể là "category_main", "subcat_1" và "subcat_2".

In [ ]:
def transform_category_name(category_name):
    try:
        main, sub1, sub2= category_name.split('/')
        return main, sub1, sub2
    except:
        return "none", "none", "none"

train['category_main'], train['subcat_1'], train['subcat_2'] = zip(*train['category_name'].apply(transform_category_name))
test['category_main'], test['subcat_1'], test['subcat_2'] = zip(*test['category_name'].apply(transform_category_name))
train.head(10)

In [ ]:
print("There are %d unique main-categories." % train['category_main'].nunique())
print("There are %d unique first sub-categories." % train['subcat_1'].nunique())
print("There are %d unique second sub-categories." % train['subcat_2'].nunique())


In [ ]:
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot


x = train['category_main'].value_counts().index.values.astype('str')
y = train['category_main'].value_counts().values
pct = [("%.2f"%(v*100))+"%"for v in (y/len(train))]

trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Main Category',
              yaxis = dict(title='Count'),
              xaxis = dict(title='Category'))
fig=dict(data=[trace1], layout=layout)
iplot(fig)

Từ biểu đồ trên, có thể nói rằng các sản phẩm dành cho 'Women' có số lượng lớn nhất. tiếp theo là các sản phẩm 'beauty'. Danh mục chung lớn thứ 3 là 'Kids'.


In [ ]:
x = train['subcat_1'].value_counts().index.values.astype('str')[:15]
y = train['subcat_1'].value_counts().values[:15]
pct = [("%.2f"%(v*100))+"%"for v in (y/len(train))][:15]

trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Main Category',
              yaxis = dict(title='Count'),
              xaxis = dict(title='subcat_1'))
fig=dict(data=[trace1], layout=layout)
iplot(fig)


Vì các sản phẩm 'women' xuất hiện với số lượng lớn trong Danh mục 'main_category', nên các danh mục xuất hiện nhiều nhất trong Danh mục 'subcat_1' phù hợp với danh các danh mục nhiều nhất ở mục 'main_category' ('Athletic Apparel','Makeup','Tops&Blueses')

In [ ]:
x = train['subcat_2'].value_counts().index.values.astype('str')[:15]
y = train['subcat_2'].value_counts().values[:15]
pct = [("%.2f"%(v*100))+"%"for v in (y/len(train))][:15]

trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Main Category',
              yaxis = dict(title='Count'),
              xaxis = dict(title='subcat_2'))
fig=dict(data=[trace1], layout=layout)
iplot(fig)


In [ ]:
print("There are %d unique brand names in the training dataset." % train['brand_name'].nunique())

In [ ]:
x = train['brand_name'].value_counts().index.values.astype('str')[:15]
y = train['brand_name'].value_counts().values[:15]
pct = [("%.2f"%(v*100))+"%"for v in (y/len(train))][:15]

trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Main Category',
              yaxis = dict(title='Count'),
              xaxis = dict(title='brand_name'))
fig=dict(data=[trace1], layout=layout)
iplot(fig)

Đa phần các mặt hàng không có thương hiệu, nhiều thứ 2 và 3 lần lượt là "PINK" và "Nike

Sẽ khó khăn hơn khi phân tích mục description vì đó là dữ liệu phi cấu trúc. Nhìn sơ qua về dữ liệu có thể nhận thấy, khi description càng dài thì mặt hàng có xu hướng giá cao hơn. Chúng ta sẽ loại bỏ tất cả các dấu câu, loại bỏ một số từ ngắn (stop words) trong tiếng Anh (như "a", "the", v.v.) và bất kỳ từ nào khác có độ dài nhỏ hơn 3

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
print("Handling categorical variables...")
le = LabelEncoder()
train.category_main = le.fit_transform(train.category_main)
train.subcat_1 = le.fit_transform(train.subcat_1)
train.subcat_2 = le.fit_transform(train.subcat_2)
train.brand_name = le.fit_transform(train.brand_name)

test.category_main = le.fit_transform(test.category_main)
test.subcat_1 = le.fit_transform(test.subcat_1)
test.subcat_2 = le.fit_transform(test.subcat_2)
test.brand_name = le.fit_transform(test.brand_name)
del le

In [ ]:
print("Text to seq process...")
from keras.preprocessing.text import Tokenizer
raw_text = np.hstack([train.item_description.str.lower(), train.name.str.lower()])

print("...")
tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)
print("...")
#chuyển văn bản thành chuỗi
train["seq_item_description"] = tok_raw.texts_to_sequences(train.item_description.str.lower())
test["seq_item_description"] = tok_raw.texts_to_sequences(test.item_description.str.lower())
train["seq_name"] = tok_raw.texts_to_sequences(train.name.str.lower())
test["seq_name"] = tok_raw.texts_to_sequences(test.name.str.lower())
train.shape
train.head(5)

In [ ]:
max_name_seq = np.max([np.max(train.seq_name.apply(lambda x: len(x))), np.max(test.seq_name.apply(lambda x: len(x)))])
max_seq_item_description = np.max([np.max(train.seq_item_description.apply(lambda x: len(x)))
                                   , np.max(test.seq_item_description.apply(lambda x: len(x)))])
print("max name seq "+str(max_name_seq))
print("max item desc seq "+str(max_seq_item_description))

MAX_NAME_SEQ = 10
MAX_ITEM_DESC_SEQ = 75
MAX_TEXT = np.max([np.max(train.seq_name.max())
                   , np.max(test.seq_name.max())
                  , np.max(train.seq_item_description.max())
                  , np.max(test.seq_item_description.max())])+2
MAX_CATEGORY = np.max([train.category_main.max(), test.category_main.max(), train.subcat_1.max(), test.subcat_1.max(),train.subcat_2.max(), test.subcat_2.max()])+1
MAX_BRAND = np.max([train.brand_name.max(), test.brand_name.max()])+1
MAX_CONDITION = np.max([train.item_condition_id.max(), test.item_condition_id.max()])+1


In [ ]:
train["target"] = np.log(train.price+1)
target_scaler = MinMaxScaler(feature_range=(-1, 1))
train["target"] = target_scaler.fit_transform(train.target.values.reshape(-1,1))
pd.DataFrame(train.target).hist()
from sklearn.model_selection import train_test_split
dtrain, dvalid = train_test_split(train, random_state=123, train_size=0.70)
print(dtrain.shape)
print(dvalid.shape)

In [ ]:

from keras.preprocessing.sequence import pad_sequences

def get_keras_data(dataset):
    X = {
        'name': pad_sequences(dataset.seq_name, maxlen=MAX_NAME_SEQ)
        ,'item_desc': pad_sequences(dataset.seq_item_description, maxlen=MAX_ITEM_DESC_SEQ)
        ,'brand_name': np.array(dataset.brand_name)
        ,'category_main': np.array(dataset.category_main)
        ,'subcat_1': np.array(dataset.subcat_1)
        ,'subcat_2': np.array(dataset.subcat_2)
        ,'item_condition': np.array(dataset.item_condition_id)
        ,'num_vars': np.array(dataset[["shipping"]])
    }
    return X

X_train = get_keras_data(dtrain)
X_valid = get_keras_data(dvalid)
X_test = get_keras_data(test)

In [ ]:
from keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GRU, Embedding, Flatten, BatchNormalization, Conv1D, GlobalMaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import backend as K

def get_model():  
    
    #Inputs
    name = Input(shape=[X_train["name"].shape[1]], name="name")
    item_desc = Input(shape=[X_train["item_desc"].shape[1]], name="item_desc")
    brand_name = Input(shape=[1], name="brand_name")
    category_main = Input(shape=[1], name="category_main")
    subcat_1 = Input(shape=[1], name="subcat_1")
    subcat_2 = Input(shape=[1], name="subcat_2")
    item_condition = Input(shape=[1], name="item_condition")
    num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")
    
    #Embeddings layers
    emb_name = Embedding(MAX_TEXT, 50)(name)  
    emb_item_desc = Embedding(MAX_TEXT, 50)(item_desc)
    emb_brand_name = Embedding(MAX_BRAND, 10)(brand_name)
    emb_category_main = Embedding(MAX_CATEGORY, 10)(category_main)
    emb_subcat_1 = Embedding(MAX_CATEGORY, 10)(subcat_1)
    emb_subcat_2 = Embedding(MAX_CATEGORY, 10)(subcat_2)
    emb_item_condition = Embedding(MAX_CONDITION, 5)(item_condition)
    
    convs1 = []
    convs2 = []
    
    for filter_length in [1,2]:
        cnn_layer1 = Conv1D(filters=50, kernel_size=filter_length, padding='same', activation='relu', strides=1) (emb_name)
        cnn_layer2 = Conv1D(filters=50, kernel_size=filter_length, padding='same', activation='relu', strides=1) (emb_item_desc)
        
        maxpool1 = GlobalMaxPooling1D() (cnn_layer1)
        maxpool2 = GlobalMaxPooling1D() (cnn_layer2)
        
        convs1.append(maxpool1)
        convs2.append(maxpool2)

    convs1 = concatenate(convs1)
    convs2 = concatenate(convs2)
    
      #rnn layer
    
    rnn_layer1 = GRU(16) (emb_item_desc)  #GRU, sử dụng cho dữ liệu text chuyển sang chuỗi (số)
    rnn_layer2 = GRU(8) (emb_name)  # dữ liệu cho lớp tiếp theo
    
    #Liên kết các lớp của mạng NN
    
    main_l = concatenate([
        Flatten() (emb_brand_name)
        , Flatten() (emb_category_main)
        , Flatten() (emb_subcat_1)
        , Flatten() (emb_subcat_2)
        , Flatten() (emb_item_condition)
        , convs1 
        , convs2
        , rnn_layer1
        , rnn_layer2
        , num_vars
    ])
    
    main_l = Dropout(0.25)(Dense(128, activation='relu') (main_l))
    main_l = Dropout(0.1)(Dense(64, activation='relu') (main_l)) 
    
    output = Dense(1, activation='linear') (main_l)

    model = Model([name, item_desc, brand_name
                   , category_main, subcat_1, subcat_2, item_condition, num_vars], output)
    model.compile(loss='mse', optimizer='adam')
    
    return model

In [ ]:
BATCH_SIZE = 20000
epochs = 5
model = get_model()
model.summary()

In [ ]:
model.fit(X_train, dtrain.target, epochs=epochs, batch_size=BATCH_SIZE
          , validation_data=(X_valid, dvalid.target)
          , verbose=1)

In [ ]:
import math

val_preds = model.predict(X_valid)
val_preds = target_scaler.inverse_transform(val_preds)
val_preds = np.exp(val_preds)+1

#absolute_error & squared_log_error
y_true = np.array(dvalid.price.values)
y_pred = val_preds[:,0]
v_rmsle = rmsle(y_true, y_pred)
print(" RMSLE error on dev test: "+str(v_rmsle))

In [ ]:
preds = model.predict(X_test, batch_size=BATCH_SIZE)
preds = target_scaler.inverse_transform(preds)
preds = np.expm1(preds)

submission = test[["test_id"]]
submission["price"] = preds

In [ ]:
submission.shape
submission.to_csv("submission.csv", index=False)